# Comparing proportions

Let's prepare the data from thromboembolism study in a contingency table as follows:

| treatment | recurrent | no recurrence | total |
| --------- | --------- | ------------- | ----- |
| placebo   |    73     |      756      |  829  |
| Apixaban  |    14     |      826      |  840  |
| total     |    87     |     1582      | 1669  |

A contingency table displays the results of a study with a categorical outcome. Rows represent different treatments or exposures, columns represent different outcomes. Each value is an actual number of subjects.

In [1]:
import math
import numpy as np
from scipy import stats

In [3]:
table = np.array([[73, 756], [14, 826]]) # example data from page 264
# contingency table (rows = alternative treatments, cols = alternative outcomes)

In [4]:
table

array([[ 73, 756],
       [ 14, 826]])

Form a Pandas dataframe, one could obtain a contingency table e.g.:
```python
table = pd.crosstab(
	data['ed'].dropna()>2, # attention, np.nan > 2 gives False
	data['dep.cons'],
	margins=True) # margins are 'totals'
```

##  Fisher's test

The goal of the analysis is to generalize about the general population of patients who have had a thromboembolism by computing CIs and a P value. The P value answers the question
>if H0 were true, which is that the treatment **does not alter the risk** of recurrence, what is the chance that random sampling of subjects would result in incidence rates as different or more different from what we observe?

Use chi-square for large sample sizes or more than 2 outcomes.

In [7]:
odds_ratio, p_value = stats.fisher_exact(table)

print("Odds ratio\t", odds_ratio)
print("P value\t\t", p_value)

Odds ratio	 5.697089947089947
P value		 1.3340996598862038e-11


### The attributable risk

One way to summarize the results is to calculate the **difference between the two proportions** also called **attributabel risk**.

In [8]:
attributable_risk = -np.diff(table[:,0]/(table[0].sum()))[0]
print(f"attributable risk = {100*attributable_risk:.1f}%")

attributable risk = 7.1%


### Number needed to treat

This value, i.e. the reciprocal of the attributable risk, tells you how many patients would require treatment to reduce the expected nulber of case of a defined end point by one.
Below, for every 14 patients who receive the treatment you'd expect to prevent a recurrent disease in one; the other 13 would receive no benefit from treatment.

In [9]:
print(f"NTT = {1/attributable_risk:.0f}")

NTT = 14


### Relative Risk

The **Risk Ratio** or **Relative Risk** is 8.8/1.7 i.e. 5.2. Subjects treated with placebo were 5.2 times as likely as treated patients to have a recurrent disease.
When we flip the ratio,  1.7/8.8 = 0.19, patients receiving the drug had 19% the risk of a recurrent disease than did patients receiving placebo. Beware which group is set as 'control' to interpret the RR. Here placebo is considered as 'risk' compared to the treatment with anticoagulant. When the relative risk associated with a factor is less than 1, then the factor is called a _protective factor_. Here the drug lowered the _absolute risk_ by 7.1% and reduced the _relative risk_ by 81%.

$$\text{Risk Ratio} = \frac{\text{EE}}{\text{EE}+\text{EN}}/\frac{\text{CE}}{\text{CE}+\text{CN}}$$

with

| treatment | recurrent | no recurr |
| --------- | --------- | --------- |
| placebo   |    EE     |    EN     |
| Apixaban  |    CE     |    CN     |


with Event in Experimental group smaller than Non-Event, same in the Control group.

In [10]:
rr = (73/829) / (14/840)
print(f"Relative Risk = {rr:.3f}")

Relative Risk = 5.283


### Odds Ratio

The Risk Ratio (Relative Risk) is different from the Odds Ratio, from which is $\text{odds} = \frac{p}{1-p}$:

$$\text{OR} = \frac{\text{EE}/\text{EN}}{\text{CE}/\text{CN}}$$

In [11]:
print(f"Odds ratio = {(73/756)/(14/826):.3f}")

Odds ratio = 5.697


## Confidence intervals

### 95% CI of the proportions

Computing the CI using the modified Wald method by hand as follows:

1. $p'=\frac{s+2}{n+4}$ with $s$ the number of successes and $n$ the number of trials
2. The margin of error is: $W=2\sqrt{\frac{p'(1-p')}{n+4}}$
3. Finally, the 95% CI is from $(p'-W)$ to $(p'+W)$

In [12]:
s,n = 73,829
p = (s+2)/(n+4)
W = 2 * (p * (1-p) / (n+4))**.5
print(f"The 95% CI for the {s}/{n} proprotion, i.e. {100*s/n:.1f}%, is {100*(p-W):.1f}% to {100*(p+W):.1f}% acc. Wald method.")

The 95% CI for the 73/829 proprotion, i.e. 8.8%, is 7.0% to 11.0% acc. Wald method.


In [13]:
s,n = 14,840
p = (s+2)/(n+4)
W = 2 * (p * (1-p) / (n+4))**.5
print(f"The 95% CI for the {s}/{n} proprotion, i.e. {100*s/n:.1f}%, is {100*(p-W):.1f}% to {100*(p+W):.1f}% acc. Wald method.")

The 95% CI for the 14/840 proprotion, i.e. 1.7%, is 1.0% to 2.8% acc. Wald method.


### 95% CI of the attributable risk

With the attributable risk (AR) = p1 - p2, the confidence interval is computing as follows:

$$\text{CI} = \text{AR} \pm 1.96 \times \sqrt{p \times (1-p) \times (1/(\text{EE}+\text{EN}) + 1/(\text{CE}+\text{CN}))}$$

where $p$ the risk, i.e. the number of events (EE) divided by the grand total.

In [14]:
ar = attributable_risk # as computed earlier
p = 73 / 1669 # risk is placebo
factor_ntt = 1.96 * np.sqrt(p * (1-p) * (1/829 + 1/840))

print(f"CI for the AR {100*attributable_risk:.1f}% extends from {100*(ar - factor_ntt):.1f}% to {100*(ar + factor_ntt):.1f}%")

CI for the AR 7.1% extends from 5.2% to 9.1%


### CI of the NNT

The CI of the NNT is obtained by taking the reciprocal of each end of the CI of the AR.

In [15]:
print(f"CI for the NTT {1/attributable_risk:.1f} extends from {1/(ar + factor_ntt):.1f} to {1/(ar - factor_ntt):.1f}")

CI for the NTT 14.1 extends from 11.0 to 19.4


### CI of the relative risk

We use the following formula:

$$\text{CI} = \exp(\ln(\text{RR}) \pm Z_{\alpha/2} \times \sqrt{1/\text{EE} \times \frac{\text{EN}}{\text{EE}+\text{EN}} + 1/\text{CE} \times \frac{\text{CN}}{\text{CE}+\text{CN}}})$$

For tables with high numnber of events and 95% CI, $Z_{\alpha/2} \approx 1.96$

In [16]:
conf=0.95
t = stats.t(df = (73+756+14+826) - 1).ppf((1 + conf) /2)
factor = t * np.sqrt((1/73 * 756/829) + (1/14 * 826/840))
ci = round(np.exp(np.log(rr) - factor), 3), round(np.exp(np.log(rr) + factor), 3)

print(f"CI for the AR {rr:.3f} extends from {ci[0]:.3f} to {ci[1]:.3f}")

CI for the AR 5.283 extends from 3.005 to 9.288


### CI of the odds ratio

To calculate the confidence interval, we use the log odds ratio, $ln(\text{OR})$, and calculate its standard error:
$ \sqrt{\frac{1}{\text{EE}} + \frac{1}{\text{EN}} + \frac{1}{\text{CE}} + \frac{1}{\text{CN}}}$;  
and then CI = $\exp(\ln(OR) \pm Z_{\alpha/2}*\text{SEM}(OR))$

In [17]:
conf=0.95
se_or = (1/73 + 1/756 + 1/14 + 1/826)**.5
t = stats.t(df = (73+756+14+826) - 1).ppf((1 + conf) /2)
W = t*se_or
ci = round(np.exp(np.log(odds_ratio) - W), 3), round(np.exp(np.log(odds_ratio) + W), 3)
print(ci)

(3.188, 10.183)


## statsmodels Table2x2()

All the estimates and statistics, including the lower and upper confidence limits of the CIs, can be obtained using the statsmodels Table2x2 method.

The p-value is based on a Wald-type z-test for the standard estimate of the log odds ratio. It uses the asymptotic approximation for the standard error. It is a large-sample test. Testing log odds = 0 is the same as testing for independence of rows/columns using the chi^2 test, difference of conditional proportions test, the likelihood rato test, or Fisher's exact test. These tests are all different in finite samples, but asymptotically equivalent.

In [18]:
import statsmodels.api as sm

table = sm.stats.Table2x2(
    [
        [73, 756],
        [14, 826]
    ]
)

In [19]:
table.summary(method='normal')

,Estimate,SE,LCB,UCB,p-value
Odds ratio,5.697,,3.189,10.178,0.000
Log odds ratio,1.740,0.296,1.160,2.320,0.000
Risk ratio,5.283,,3.007,9.284,0.000
Log risk ratio,1.665,0.288,1.101,2.228,0.000


## Chi²

### Comparing observed and expected proportions

We take the example of the Mendel's peas experiment:

| phenotype       | observed number | expected prop | expected number |
|:--------------- |:---------------:|:-------------:|:---------------:|
| round,   yellow |       315       |      9/16     |      312.75     |
| round,   green  |       108       |      3/16     |      104.25     |
| angular, yellow |       101       |      3/16     |      104.25     |
| angular, green  |        32       |      1/16     |       34.75     |
| TOTAL           |       556       |     16/16     |      556.00     |

Is the discrepancy between the observed and expected distributions greater that we would expect to see by chance?

To quantify the discrepancy between the observed and expected distributions, the chi-square test does the following for each category:

$$\chi^2 = \sum \frac{(\text{observed} - \text{expected})^2}{\text{expected}} $$

The P value answers the question
>if the theory that generated the expected distribution is correct, what is the chance that random sampling would lead to a **deviation from expected as large or larger as that observed** in this experiment?

In [20]:
stats.chisquare(f_obs=[315, 108, 101, 32], f_exp=[312.75, 104.25, 104.25, 34.75])

Power_divergenceResult(statistic=0.4700239808153477, pvalue=0.925425895103616)

With such a large P value, the deviation from the theory are small and consistent with random variation. The number of degrees of freedom equals the number of categories minus one. This test is based on some approximations.

When there are only two categories, the _binomial test_ computes the exact P values without any approximation or worry about the sample size.

In [21]:
from statsmodels.stats.proportion import proportions_chisquare

count = np.array([73, 14])
nobs = np.array([829, 840])

# note that in this case (a two sample case with two sided
# alternative), the test produces the same value as porportions_ztest
# since the chi-square distribution is the square of a normal distribution
chisq, pvalue, table = proportions_chisquare(count, nobs)
print(f"chisq = {chisq:.2f}, pvalue = {pvalue:.3f}")

chisq = 43.04, pvalue = 0.000


The $\chi^2$ statistic has an expected distribution that can be used to determine the p-value associated with this statistic. As with the $t$-distribution, the $\chi^2$-distribution depends on the degrees of freedom. This is calculated by subtracting one from the number of rows and from the number of columns in the contingency table and multiplying them together. In the case below we have 2 rows and 2 columns, which calculates to 1 degrees of freedom. Armed with our $\chi^2$ statistic and our degrees of freedom, we can now calculate the p-value for the hypothesis test.

In [22]:
# with low P value we can reject H0 that data follow/are consistent with the theoritical distribution
chi2, p, dof, expected = stats.chi2_contingency([[73, 756], [14, 826]])
print(f"chi² = {chi2:.3f}")
print(f"P value = {p:.5f}")
print(f"with {dof} degrees of freedom")

chi² = 41.605
P value = 0.00000
with 1 degrees of freedom


In [23]:
expected

array([[ 43.21330138, 785.78669862],
       [ 43.78669862, 796.21330138]])

The observed values must be an integer, but the expected value, averaged over many experiments, can be a fraction. $\chi^2$ is the statistic returned by the test.

## Case-control studies

Two groups of subjects:
- incident **cases**, with a **disease** or condition
- **controls**, selected from the same population that produced the cases

For example we can ask whether people with cholera are less likely to have been vaccinated than those who have not gotten cholera. We set the column totals by choosing how many cases and controls top study, and then determined whether each person had been previously vaccinated.

| vaccine | cases | controls |
| ------- |:-----:|:--------:|
| yes     |  10   |    94    |
| no      |  33   |    78    |
| total   |  43   |   172    |

Computing risks and the relative risk directly from a case-control study is invalid because it depends on the total number of controls.

Results from a case-control study are summarized as an **odds ratio** which compares the **odds of exposure** (vaccination) in cases and controls. _The odds ratio equals the odds of the cases having been exposed to the possible risk factor, divided by the odds of the controls having been exposed_. Below the odds ratio of 0.25 quantifies the reduced odds of cholera among those who are vaccinated. We can conclude that a vaccinated person has 25% the chance of an unvaccinated person getting disease, or the vaccine is 75% effective in preventing disease.

The Odds Ratio from a case-control study is almost always a good estimate of the Relative Risk. The Odds Ratio can be only interpret as Relative Risks when the **disease is rare**. But in fact OR can be interpreted as RR in most but not all case-control studies even those of common diseases.

In [22]:
print(f"Odds ratio = {(10/33)/(94/78):.3f}")

Odds ratio = 0.251


In [23]:
sm.stats.Table2x2(
    [
        [10, 94],
        [33, 78]
    ]
).summary(method='normal')

,Estimate,SE,LCB,UCB,p-value
Odds ratio,0.251,,0.117,0.542,0.000
Log odds ratio,-1.381,0.392,-2.149,-0.612,0.000
Risk ratio,0.323,,0.168,0.623,0.001
Log risk ratio,-1.129,0.334,-1.784,-0.474,0.001


### Another example

In [24]:
sm.stats.Table2x2(
    [
        [  25,   213],
        [1935, 19216]
    ]
).summary(method='normal')

,Estimate,SE,LCB,UCB,p-value
Odds ratio,1.166,,0.768,1.769,0.471
Log odds ratio,0.153,0.213,-0.264,0.570,0.471
Risk ratio,1.148,,0.791,1.668,0.468
Log risk ratio,0.138,0.190,-0.235,0.511,0.468


In the example above, the CI contains 1.0, so we conclude that there is no evidence of an association between the treatment and the outcome (cases)